# Generate FAQ for README

Let's get started with pandas-chatflow!!

In [1]:
import json
from openai import OpenAI
from openaivec import pandas_ext
from pydantic import BaseModel, Field
import pandas as pd

pandas_ext.use(OpenAI())

pandas_ext.responses_model("gpt-4.1")

The model name 'gpt-4.1' is not supported by tiktoken. Instead, using the 'o200k_base' encoding.


In [2]:
docs_df: pd.DataFrame = pd.DataFrame(
    {"title": "readme", "body": [open("../../README.md").read()]}
)
docs_df

,title,body
0,readme,# What is this?\n\n**openaivec** is a Python l...


In [3]:
class Question(BaseModel):
    question: str = Field(description="The question to ask the model.")
    answer: str = Field(description="The answer to the question.")

class Section(BaseModel):
    title: str = Field(description="The title of the section.")
    content: str = Field(description="The content of the section.")
    questions: list[Question] = Field(description="List of questions and answers related to the section.")

class Document(BaseModel):
    sections: list[Section] = Field(description="List of sections in the document.")

In [4]:
sections_df = docs_df.pipe(
    lambda df: df
    .assign(
        section=lambda df: df["body"].ai.responses(
            instructions="""
            Generate a list of FAQ for each section of the document. 
            Break down the document into as many detailed sections as possible, 
            regardless of markdown format.
            """,
            response_format=Document
        )
        .map(lambda x: x.sections)
    )
    .drop(columns=["body"])
    .explode("section")
    .ai.extract("section")
)

sections_df


,title,section_title,section_content,section_questions
0,readme,Introduction to openaivec,openaivec is a Python library for efficient te...,"[{'question': 'What is openaivec?', 'answer': ..."
0,readme,Generative Mutation for Tabular Data,You can mutate columns in Pandas Series or Dat...,[{'question': 'How do I use openaivec to trans...
0,readme,Overview and Features,openaivec provides a vectorized interface for ...,[{'question': 'What is the main advantage of o...
0,readme,Requirements and Installation,Requires Python 3.10 or higher. Install with p...,[{'question': 'What Python version is required...
0,readme,Basic Usage,Initialize the vectorized client with your Ope...,[{'question': 'How do I initialize the vectori...
0,readme,Using with Pandas DataFrame,openaivec.pandas_ext extends pandas.Series wit...,[{'question': 'How do I enable openaivec for P...
0,readme,Using with Apache Spark UDF,You can create UDFs for Spark using UDFBuilder...,[{'question': 'How do I create a Spark UDF wit...
0,readme,Building Prompts with FewShotPromptBuilder,FewShotPromptBuilder helps you create few-shot...,"[{'question': 'What is FewShotPromptBuilder?',..."
0,readme,Improving Prompts with OpenAI,FewShotPromptBuilder's improve method uses Ope...,"[{'question': 'How do I improve a prompt?', 'a..."
0,readme,Using with Microsoft Fabric,Instructions for integrating openaivec with Mi...,[{'question': 'How do I add openaivec to Micro...


In [5]:
questions_df: pd.DataFrame = sections_df.pipe(
    lambda df: df
    .drop(columns=["section_content"])
    .explode("section_questions")
    .ai.extract("section_questions")
    .reset_index(drop=True)
)


In [6]:
from IPython.display import Markdown, display

display(Markdown(questions_df.to_markdown()))

|    | title   | section_title                              | section_questions_question                                      | section_questions_answer                                                                                                            |
|---:|:--------|:-------------------------------------------|:----------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------|
|  0 | readme  | Introduction to openaivec                  | What is openaivec?                                              | A Python library for efficient text processing using the OpenAI API, with integration for Pandas and Apache Spark.                  |
|  1 | readme  | Introduction to openaivec                  | What can I do with openaivec?                                   | You can generate embeddings, text responses, and perform other language model tasks directly within your data processing workflows. |
|  2 | readme  | Introduction to openaivec                  | Where can I find the full API reference?                        | At https://openaivec.anareg.design/.                                                                                                |
|  3 | readme  | Generative Mutation for Tabular Data       | How do I use openaivec to translate a Pandas Series?            | Use the ai.responses accessor, e.g., animals.ai.responses('Translate the animal names to Chinese.').                                |
|  4 | readme  | Generative Mutation for Tabular Data       | Can I use openaivec to add multiple columns to a DataFrame?     | Yes, you can use df.assign with lambda functions that call ai.responses for each new column.                                        |
|  5 | readme  | Generative Mutation for Tabular Data       | What are some example tasks I can perform?                      | Translation, color assignment, and technical word classification.                                                                   |
|  6 | readme  | Overview and Features                      | What is the main advantage of openaivec's vectorized interface? | It allows processing multiple inputs in a single API call, reducing latency and simplifying code.                                   |
|  7 | readme  | Overview and Features                      | Which platforms does openaivec integrate with?                  | Pandas DataFrames and Apache Spark UDFs.                                                                                            |
|  8 | readme  | Overview and Features                      | Is openaivec compatible with Azure OpenAI?                      | Yes, it supports multiple OpenAI clients, including Azure OpenAI.                                                                   |
|  9 | readme  | Requirements and Installation              | What Python version is required?                                | Python 3.10 or higher.                                                                                                              |
| 10 | readme  | Requirements and Installation              | How do I install openaivec?                                     | Run pip install openaivec.                                                                                                          |
| 11 | readme  | Requirements and Installation              | How do I uninstall openaivec?                                   | Run pip uninstall openaivec.                                                                                                        |
| 12 | readme  | Basic Usage                                | How do I initialize the vectorized client?                      | Create a VectorizedResponsesOpenAI instance with your OpenAI client and desired parameters.                                         |
| 13 | readme  | Basic Usage                                | How do I process a list of inputs?                              | Call the parse method with your list, e.g., client.parse(['panda', 'rabbit', 'koala']).                                             |
| 14 | readme  | Using with Pandas DataFrame                | How do I enable openaivec for Pandas?                           | Import pandas_ext and call pandas_ext.use(OpenAI()).                                                                                |
| 15 | readme  | Using with Pandas DataFrame                | How do I set models for responses and embeddings?               | Use pandas_ext.responses_model and pandas_ext.embeddings_model.                                                                     |
| 16 | readme  | Using with Pandas DataFrame                | How do I use ai.responses in a DataFrame?                       | Use df.assign with a lambda that calls df.column.ai.responses(prompt).                                                              |
| 17 | readme  | Using with Apache Spark UDF                | How do I create a Spark UDF with openaivec?                     | Use UDFBuilder.of_azureopenai with your API details, then register the UDF with spark.udf.register.                                 |
| 18 | readme  | Using with Apache Spark UDF                | What are some example UDF tasks?                                | Extracting flavor or product type from product names.                                                                               |
| 19 | readme  | Using with Apache Spark UDF                | How do I use the UDFs in Spark SQL?                             | Call them in your SELECT statements, e.g., parse_taste(product_name).                                                               |
| 20 | readme  | Building Prompts with FewShotPromptBuilder | What is FewShotPromptBuilder?                                   | A class to help build few-shot learning prompts for LLMs.                                                                           |
| 21 | readme  | Building Prompts with FewShotPromptBuilder | How do I use it?                                                | Set the purpose, cautions, and examples, then call build().                                                                         |
| 22 | readme  | Building Prompts with FewShotPromptBuilder | What format does the prompt output use?                         | XML format.                                                                                                                         |
| 23 | readme  | Improving Prompts with OpenAI              | How do I improve a prompt?                                      | Call the improve method with your OpenAI client, model name, and max_iter.                                                          |
| 24 | readme  | Improving Prompts with OpenAI              | What does the improve method do?                                | It refines the prompt to eliminate contradictions, ambiguities, and redundancies.                                                   |
| 25 | readme  | Using with Microsoft Fabric                | How do I add openaivec to Microsoft Fabric?                     | Create an environment, add openaivec from PyPI to the custom library, and use it in your notebook.                                  |
| 26 | readme  | Using with Microsoft Fabric                | Can I use openaivec.spark.UDFBuilder in Microsoft Fabric?       | Yes, after setting up the environment and installing the library.                                                                   |
| 27 | readme  | Contributing                               | How can I contribute to openaivec?                              | Fork the repository, create a branch, add tests, ensure tests pass, and lint your code.                                             |
| 28 | readme  | Contributing                               | How do I install development dependencies?                      | Run uv sync --all-extras --dev.                                                                                                     |
| 29 | readme  | Contributing                               | How do I reformat the code?                                     | Run uv run ruff check . --fix.                                                                                                      |
| 30 | readme  | Community                                  | Is there a community for openaivec?                             | Yes, you can join the Discord community for developers.                                                                             |

In [7]:
ja_questions_df: pd.DataFrame = questions_df.pipe(
    lambda df: df
    .ai.responses(
        instructions="""
        Translate given json into japanese with same schema.
        Just return the json without any additional text.
        """
    )
    .map(json.loads)
    .ai.extract()
)

In [8]:
display(Markdown(ja_questions_df.to_markdown()))

|    | record_title   | record_section_title                     | record_section_questions_question                           | record_section_questions_answer                                                                              |
|---:|:---------------|:-----------------------------------------|:------------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------|
|  0 | readme         | openaivecの紹介                          | openaivecとは何ですか？                                     | OpenAI APIを利用した効率的なテキスト処理のためのPythonライブラリで、PandasやApache Sparkとの統合が可能です。 |
|  1 | readme         | openaivecの紹介                          | openaivecで何ができますか？                                 | 埋め込み生成、テキスト応答、その他の言語モデルタスクをデータ処理ワークフロー内で直接実行できます。           |
|  2 | readme         | openaivecの紹介                          | APIリファレンスの全体はどこで見られますか？                 | https://openaivec.anareg.design/ でご覧いただけます。                                                        |
|  3 | readme         | 表データの生成的変換                     | PandasのSeriesを翻訳するにはどうすればよいですか？          | ai.responsesアクセサを使用します。例：animals.ai.responses('動物名を中国語に翻訳してください。')             |
|  4 | readme         | 表データの生成的変換                     | openai vecでDataFrameに複数の列を追加できますか？           | はい、df.assignと、各新しい列に対してai.responsesを呼び出すlambda関数を使うことができます。                  |
|  5 | readme         | 表データの生成的変換                     | どんなタスクの例がありますか？                              | 翻訳、色の割り当て、技術用語の分類などです。                                                                 |
|  6 | readme         | 概要と特徴                               | openaivecのベクトル化インターフェースの主な利点は何ですか？ | 複数の入力を1回のAPIコールで処理でき、レイテンシを削減しコードを簡素化できます。                             |
|  7 | readme         | 概要と特徴                               | openaivecはどのプラットフォームと統合できますか？           | Pandas DataFrameとApache Spark UDFに対応しています。                                                         |
|  8 | readme         | 概要と特徴                               | openaivecはAzure OpenAIと互換性がありますか？               | はい、Azure OpenAIを含む複数のOpenAIクライアントをサポートしています。                                       |
|  9 | readme         | 要件とインストール                       | 必要なPythonバージョンは？                                  | Python 3.10以上です。                                                                                        |
| 10 | readme         | 要件とインストール                       | openaivecのインストール方法は？                             | pip install openaivec を実行してください。                                                                   |
| 11 | readme         | 要件とインストール                       | openaivecのアンインストール方法は？                         | pip uninstall openaivec を実行してください。                                                                 |
| 12 | readme         | 基本的な使い方                           | ベクトル化クライアントの初期化方法は？                      | OpenAIクライアントと必要なパラメータでVectorizedResponsesOpenAIインスタンスを作成します。                    |
| 13 | readme         | 基本的な使い方                           | 入力リストを処理するには？                                  | parseメソッドにリストを渡します。例：client.parse(['panda', 'rabbit', 'koala'])                              |
| 14 | readme         | Pandas DataFrameでの利用                 | Pandasでopenaivecを有効にするには？                         | pandas_extをインポートし、pandas_ext.use(OpenAI())を呼び出します。                                           |
| 15 | readme         | Pandas DataFrameでの利用                 | 応答や埋め込みのモデルを設定するには？                      | pandas_ext.responses_modelとpandas_ext.embeddings_modelを使用します。                                        |
| 16 | readme         | Pandas DataFrameでの利用                 | DataFrameでai.responsesを使うには？                         | df.assignで、df.column.ai.responses(prompt)を呼ぶlambdaを使います。                                          |
| 17 | readme         | Apache Spark UDFでの利用                 | openaivecでSpark UDFを作成するには？                        | UDFBuilder.of_azureopenaiにAPI情報を渡し、spark.udf.registerでUDFを登録します。                              |
| 18 | readme         | Apache Spark UDFでの利用                 | UDFタスクの例は？                                           | 商品名からフレーバーや商品タイプを抽出するなどです。                                                         |
| 19 | readme         | Apache Spark UDFでの利用                 | Spark SQLでUDFを使うには？                                  | SELECT文で呼び出します。例：parse_taste(product_name)                                                        |
| 20 | readme         | FewShotPromptBuilderによるプロンプト作成 | FewShotPromptBuilderとは？                                  | LLMのfew-shot学習用プロンプトを作成するためのクラスです。                                                    |
| 21 | readme         | FewShotPromptBuilderによるプロンプト作成 | 使い方は？                                                  | 目的、注意点、例を設定し、build()を呼び出します。                                                            |
| 22 | readme         | FewShotPromptBuilderによるプロンプト作成 | プロンプト出力のフォーマットは？                            | XMLフォーマットです。                                                                                        |
| 23 | readme         | OpenAIによるプロンプト改善               | プロンプトを改善するには？                                  | OpenAIクライアント、モデル名、max_iterを指定してimproveメソッドを呼びます。                                  |
| 24 | readme         | OpenAIによるプロンプト改善               | improveメソッドは何をしますか？                             | 矛盾、曖昧さ、冗長性を排除するようにプロンプトを洗練します。                                                 |
| 25 | readme         | Microsoft Fabricでの利用                 | Microsoft Fabricにopenaivecを追加するには？                 | 環境を作成し、PyPIからopenaivecをカスタムライブラリに追加し、ノートブックで使用します。                      |
| 26 | readme         | Microsoft Fabricでの利用                 | Microsoft Fabricでopenaivec.spark.UDFBuilderは使えますか？  | はい、環境をセットアップしライブラリをインストールした後に利用できます。                                     |
| 27 | readme         | コントリビューション                     | openaivecに貢献するには？                                   | リポジトリをフォークし、ブランチを作成、テストを追加、テストが通ることを確認し、コードをリントしてください。 |
| 28 | readme         | コントリビューション                     | 開発用依存関係のインストール方法は？                        | uv sync --all-extras --dev を実行してください。                                                              |
| 29 | readme         | コントリビューション                     | コードの再フォーマット方法は？                              | uv run ruff check . --fix を実行してください。                                                               |
| 30 | readme         | コミュニティ                             | openaivecのコミュニティはありますか？                       | はい、開発者向けのDiscordコミュニティに参加できます。                                                        |